# Parameters

In [1]:
year = 2022
week = 5
#path_so = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\Info_POS_clientes\Sell Out - No B2b\Disval'
path_so = r'C:\Users\amartinezgo\genommalabinternacional\Francisco Jose Delfino - Info_POS_clientes\Sell Out - No B2b\Disval'
#path_inv = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\Info_POS_clientes\Stocks\{0}\Semana {1}'
path_inv = r'C:\Users\amartinezgo\genommalabinternacional\Francisco Jose Delfino - Info_POS_clientes\Stocks\{0}\Semana {1}'
path_inv = path_inv.format(year, str(week).zfill(2))

# Libraries

In [2]:
import pandas as pd
from datetime import datetime
import pyodbc
import shutil
import os
from openpyxl import load_workbook

# Weekly data

## Connections to DWH

In [3]:
conn1 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_DWH;'
                     'Trusted_Connection=yes;')

conn2 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_MasterOp;'
                     'Trusted_Connection=yes;')

query_days = '''SELECT TmpFecha
                FROM Gnm_DWH.dbo.Dim_Tiempo
                WHERE TmpAnioSemanaGenomma = {0} AND TmpSemanaAnioGenomma = {1}'''

query_sucs = '''SELECT DISTINCT SUC.SucCodCliente, SUC.SucId
                    FROM (SELECT DISTINCT PaisNombre, CadID FROM Gnm_MasterOp.dbo.vw_EstructuraClientesSegPTVTotal
                    WHERE PaisNombre = 'Argentina') AS CLIE
                LEFT JOIN (
                    SELECT DISTINCT SucId, SucCodCliente, CadId FROM Gnm_MasterOp.dbo.vw_EstructuraSucursalesTotal) AS SUC
                ON CLIE.CadID = SUC.CadID 
                '''

## Stocks

### Load data

In [4]:
file_name = '{0}_{1}_Disval_INV.txt'

df_stock = pd.DataFrame()
# Dates
df_days = pd.read_sql(query_days.format(year, week), conn1)

# Import data
df = pd.read_csv(path_inv + '\\' + file_name.format(year, str(week).zfill(2)), header=None)

# Assign date into the dataframe
df['Fecha_Stock'] = df_days['TmpFecha'].max()
df_stock = pd.concat([df_stock, df])

### Clean Data

In [5]:
# Formatting data stocks
df_stock.rename(dict(zip(range(5), ['Cod_Cliente', 'EAN o UPC Genomma', 'Descripcion_Prod', 'Descripcion_Local', 'Unidades'])), axis=1, inplace=True)
df_stock['Cadena'] = 'Disval'
df_stock['Cod_Prod'] = df_stock['EAN o UPC Genomma']
df_stock['Cod_Local'] = '89310'
data_stock = df_stock[['Fecha_Stock', 'Cadena', 'Cod_Prod', 'EAN o UPC Genomma', 'Descripcion_Prod', 'Cod_Local', 'Descripcion_Local', 'Unidades']].copy()
data_stock = data_stock[~data_stock['EAN o UPC Genomma'].isin([999999, 9999999999999])].copy()
# Order columns
final_stock = data_stock[['Fecha_Stock', 'Cadena', 'Cod_Prod', 'EAN o UPC Genomma', 'Descripcion_Prod', 'Cod_Local', 'Descripcion_Local', 'Unidades']].copy()

In [6]:
final_stock.isnull().sum()

Fecha_Stock          0
Cadena               0
Cod_Prod             0
EAN o UPC Genomma    0
Descripcion_Prod     0
Cod_Local            0
Descripcion_Local    0
Unidades             0
dtype: int64

## Sell Out

In [7]:
# Select elements in intersection
def dates_repeted(dates, new_dates):
    pattern = set(dates)
    return [x for x in new_dates if x in dates]

In [8]:
# Convert date to str
def date_to_str(date):
    try:
        return date.strftime('%Y-%m-%d')
    except:
        return date

### Load data

In [9]:
%%time
# The SO data is accumulated, so we import several dates
df_so = pd.DataFrame(columns=range(9))
for year, week in [(year, w) for w in range(week - 4, week + 1)]: 
    try:
        # Search the files in differents locations
        file_name = '{0}_{1}_Disval_SO.txt'
        df = pd.read_csv(path_so + '\\' + file_name.format(year, str(week).zfill(2)), header=None)
        # Clean date
        df[5] = df[5].map(date_to_str)
        # Get the unique days
        dr = dates_repeted(df_so[5].unique(), df[5].unique())
        if len(dr) > 0:
            df_so = df_so[~df_so[5].isin(dr)].copy()
        df_so = pd.concat([df_so, df], axis=0)
    except:
        print('Ojo con el archivo ' + str(year) + ' - ' + str(week).zfill(2))

Wall time: 1.25 s


### Clean data

In [10]:
# Formatting sell out columns
df_so['Fecha'] = df_so[5].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
# Filter the specific week
df_so = df_so[df_so['Fecha'].isin(df_days['TmpFecha'].tolist())].copy()
df_so.rename({6:'EAN o UPC Genomma', 7:'Descripcion_Local', 8:'Unidades'}, axis=1, inplace=True)
df_so['Cadena'] = 'Disval'
df_so['Cod_Prod'] = df_so['EAN o UPC Genomma']
df_so['Cod_Local'] = '89310'
data_so = df_so[['Fecha', 'Cadena', 'Cod_Prod', 'EAN o UPC Genomma', 'Cod_Local', 'Descripcion_Local', 'Unidades']].copy()
# Filter EAN
data_so = data_so[~data_so['EAN o UPC Genomma'].isin([999999, 9999999999999])].copy()
# Merge with stock's Descripcion
aux = data_stock[['EAN o UPC Genomma', 'Descripcion_Prod']]
data_so = data_so.merge(aux.drop_duplicates(), on='EAN o UPC Genomma', how='left')
# Order columns
final_so = data_so[['Fecha', 'Cadena', 'Cod_Prod', 'EAN o UPC Genomma', 'Descripcion_Prod', 'Cod_Local', 'Descripcion_Local', 'Unidades']].copy()

In [11]:
final_so.isnull().sum()

Fecha                0
Cadena               0
Cod_Prod             0
EAN o UPC Genomma    0
Descripcion_Prod     0
Cod_Local            0
Descripcion_Local    0
Unidades             0
dtype: int64

# Export file

In [12]:
#Export to layout file
def write_layout(filename, df, sellout=True):
    writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a')
    writer.book = load_workbook(filename)
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    if sellout == True:
        df.to_excel(writer, sheet_name='Ventas', startcol=1, startrow=18, index=False, header=None)
    else:
        df.to_excel(writer, sheet_name='Stock', startcol=1, startrow=18, index=False, header=None)
    writer.save()

Toma el layout de ISV y crea la carpeta correspondiente:

In [13]:
original = '../../../../1Data/2Catalogue/toISV/Layout/Formato No B2B Orig.xlsx'
new_folder = '../../../../1Data/2Catalogue/toISV/{0}/S{1}'.format(year, str(week).zfill(2))

#Create a new directory
if not os.path.exists(new_folder):
    os.mkdir(new_folder)

In [14]:
%%time
filename = 'Formato No B2B Disval Sem {}'.format(str(week).zfill(2))
path_export = new_folder + '/' + filename +'.xlsx'

## Copy layout with name <filename>
shutil.copy(original, path_export)

## Export in different sheets
write_layout(path_export, final_so, True)
write_layout(path_export, final_stock, False)

Wall time: 6.61 s
